In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc",requested_cpu=4)

Vocabulary will not be loaded from file.



In [3]:
semiotic.vocab.build_str_par(vocab_size = 10, parallel=True, corpus_length=800000)

Computing in parallel
Normalize and jobs data...
... computed in 56.85757398605347 secs.

Enter loop
1/10: ('t', 'h')...
... computed in 7.40329384803772 secs.

2/10: ('i', 'n')...
... computed in 4.298695087432861 secs.

3/10: ('th', 'e')...
... computed in 4.341756820678711 secs.

4/10: ('a', 'n')...
... computed in 3.3815670013427734 secs.

5/10: ('e', 'r')...
... computed in 3.256744146347046 secs.

6/10: ('o', 'n')...
... computed in 2.8714160919189453 secs.

7/10: ('r', 'e')...
... computed in 2.6994171142578125 secs.

8/10: ('e', 'n')...
... computed in 2.182844877243042 secs.

9/10: ('a', 't')...
... computed in 2.3852827548980713 secs.

10/10: ('t', 'o')...
... computed in 2.2556779384613037 secs.

Compute freq...
... computed in 9.424883842468262 secs.

Vocabulary built


In [4]:
semiotic.vocab.head(20)

[('e', 4032304),
 ('s', 3869069),
 ('a', 3267378),
 ('o', 3208924),
 ('i', 3199776),
 ('t', 2609098),
 ('l', 2482336),
 ('d', 2300969),
 ('r', 2084177),
 ('c', 1867831),
 ('u', 1680274),
 ('m', 1482412),
 ('h', 1416923),
 ('f', 1333436),
 ('p', 1231863),
 ('g', 1193988),
 ('w', 1151290),
 ('in', 1147547),
 ('the', 1119978),
 ('y', 1109577)]

In [ ]:
semiotic.vocab.build_str_par(vocab_size = 10, parallel=False, corpus_length=800000)

In [3]:
%load_ext line_profiler

In [ ]:
%lprun -f semiotic.vocab.build_str_par semiotic.vocab.build_str_par(vocab_size = -4, parallel=True, corpus_length=400000)

In [4]:
a,b = semiotic.vocab.build_par(vocab_size = -2, parallel=True)

Normalize & Alphabet in parallel...
Normalize & Alphabet computed in 0:00:02.486709
Computing matrix values in parallel...
Matrix values computed in 0:00:03.129686
Alphabet Size: 493


In [14]:
sorted(a[1].items())==sorted(b[1].items())

True

In [11]:
b[0].items()

dict_items([(7, 104238), (10, 88776), (5, 48062), (3, 32320), (6, 30577), (11, 25249), (9, 24584), (2, 23994), (14, 15531), (0, 14070), (13, 13327), (4, 10391), (15, 8707), (17, 5674), (16, 5133), (12, 4442), (18, 3383), (23, 2818), (19, 2099), (1, 1689), (22, 1202), (8, 1026), (32, 698), (30, 677), (28, 366), (34, 354), (42, 253), (25, 203), (29, 179), (21, 159), (35, 142), (38, 141), (37, 135), (33, 135), (31, 127), (36, 124), (27, 108), (48, 64), (46, 61), (24, 49), (41, 34), (20, 24), (64, 19), (47, 15), (63, 14), (58, 13), (53, 10), (135, 10), (51, 8), (112, 6), (52, 6), (55, 5), (69, 5), (92, 5), (119, 4), (125, 4), (116, 4), (79, 3), (115, 3), (107, 3), (81, 3), (76, 3), (66, 3), (67, 3), (78, 3), (43, 3), (164, 3), (39, 3), (154, 2), (97, 2), (230, 2), (108, 2), (57, 2), (54, 2), (111, 2), (217, 2), (83, 2), (26, 2), (73, 2), (106, 1), (169, 1), (363, 1), (146, 1), (123, 1), (377, 1), (60, 1), (80, 1), (91, 1), (94, 1), (65, 1), (271, 1), (104, 1), (327, 1), (138, 1), (74, 1), 

In [5]:
profiler_result = "Timer unit: 1e-06 s\n\nTotal time: 32.0482 s\nFile: /Users/Gianni/semiolog/semiolog/vocabulary.py\nFunction: build_par at line 646\n\nLine #      Hits         Time  Per Hit   % Time  Line Contents\n==============================================================\n   646                                               def build_par(\n   647                                                   self,\n   648                                                   corpus = None,\n   649                                                   vocab_size = None,\n   650                                                   special_tokens = None,\n   651                                                   save = False,\n   652                                                   save_step = None,\n   653                                                   progress_bar = True,\n   654                                                   resume_merges = False,\n   655                                                   parallel = True,\n   656                                                   sparse = True,\n   657                                                   sparse_mode = \"csr\",\n   658                                                   corpus_length = None\n   659                                                   ):\n   660                                                   \"\"\"\n   661                                                   Build vocabulary from a Corpus.\n   662                                                   Vocabularies can be extended by providing an existing merging list. If resume_merges = True, the current merges in self.merges will be used. Otherwise one can provide a list of merges as value of resume_merges.\n   663                                                   \n   664                                                   If vocab_size is negative, the value is taken, not as the target size of the voc, but as the number of new terms to compute beyond the size of the initial alphabet\n   665                                                   \"\"\"\n   666                                                   \n   667         1          9.0      9.0      0.0          if corpus == None:\n   668         1          4.0      4.0      0.0              corpus = self.name\n   669                                                   \n   670         1          4.0      4.0      0.0          if vocab_size == None:\n   671                                                       vocab_size = self.config.size\n   672                                                   \n   673         1          4.0      4.0      0.0          if special_tokens == None:\n   674         1          6.0      6.0      0.0              special_tokens = self.config.special_tokens\n   675                                                   \n   676         1          4.0      4.0      0.0          if save == True and save_step != None:\n   677                                                       saveQ = True\n   678                                                       \n   679                                                       if not isdir(self.path):\n   680                                                           makedirs(self.path)\n   681                                                           \n   682                                                       # save_steps = {save_step*i for i in range(int(abs(vocab_size)/save_step)+1)}\n   683                                                   else:\n   684         1          4.0      4.0      0.0              saveQ = False\n   685                                           \n   686         1          5.0      5.0      0.0          def parallel_chain(chain, n_of_parts, overlap = 0):\n   687                                                       \"\"\"\n   688                                                       Breaks the chain in n chunks to compute best pair of terms. \"overlap\" gives the number of terms of overlap between chunks to alow to ensure that no pair of terms is lost due to the break.\n   689                                                       \"\"\"\n   690                                                       chunk_size = int(len(chain) / n_of_parts)+1            \n   691                                                       for i in range(0, len(chain), chunk_size):\n   692                                                           yield chain[i : i + chunk_size + overlap]\n   693                                                           \n   694         1          5.0      5.0      0.0          def extract_drc(pairs, encoder: dict):\n   695                                                       data = []\n   696                                                       rows = []\n   697                                                       columns = []\n   698                                                       for (r,c),d in pairs:\n   699                                                           data.append(d)\n   700                                                           rows.append(encoder[r])\n   701                                                           columns.append(encoder[c])\n   702                                                       return data, rows, columns\n   703                                           \n   704         1          4.0      4.0      0.0          def separate_chain(chain, n_of_parts, best_pair: list):\n   705                                                       \"\"\"\n   706                                                       Separate a chain (in list form) for parallel processing of regex findall of pair, taking care that the cuts of the chunks don't fall in the neiborhood of the pair, affecting the final counts\n   707                                                       \"\"\"\n   708                                                       chunk_size = int(len(chain) / n_of_parts)+1\n   709                                                       b = 0\n   710                                                       n = chunk_size\n   711                                                       chain_len = len(chain)\n   712                                                       for i in range(n_of_parts):\n   713                                                           n = (i+1)*chunk_size\n   714                                                           if chain_len > n:\n   715                                                               while chain[n-2:n] == best_pair or chain[n-1:n+1] == best_pair:\n   716                                                                   n = n+1\n   717                                                           yield (\"[SEP_i] \" if i!=0 else \"\") + \" \".join(chain[b:n]) + (\" [SEP_i]\" if i!=n_of_parts-1 else \"\")\n   718                                                           b = n-1\n   719                                           \n   720         1          5.0      5.0      0.0          def agglutinate_chain(pair, cl_chain):\n   721                                                       bigram = re.escape(\" \".join(pair))\n   722                                                       p = re.compile(r\"(?<!\\S)\" + bigram + r\"(?!\\S)\")\n   723                                                       cl_chain = p.sub(\"\".join(pair), cl_chain)\n   724                                                       return cl_chain\n   725                                                   \n   726         1          5.0      5.0      0.0          if isinstance(self.config.normalizer,list):\n   727         2         78.0     39.0      0.0              normalizer = eval(\n   728         1          8.0      8.0      0.0                  f\"tokenizer.normalizers.Sequence({self.config.normalizer})\"\n   729                                                           )\n   730                                                   else:\n   731                                                       normalizer = eval(\n   732                                                           f\"tokenizer.normalizers.{util.if_none_disable(self.config.normalizer)}\"\n   733                                                       )\n   734                                                   \n   735         1          5.0      5.0      0.0          if parallel:\n   736                                                       \n   737         1      20404.0  20404.0      0.1              with Parallel(n_jobs=self.cpu_count) as parallel_pool:\n   738                                                           \n   739         1      33518.0  33518.0      0.1                  par_corpus = parallel_chain(self.corpus.train[:corpus_length], self.cpu_count)\n   740                                           \n   741         1       1691.0   1691.0      0.0                  print(\"Normalize & Alphabet in parallel...\")\n   742         1         57.0     57.0      0.0                  start = datetime.now()\n   743                                                           \n   744                                                           # TODO: check if chain_list_alpha could be done without for loop\n   745         1    6516506.0 6516506.0     20.3                  result = parallel_pool(delayed(self.chain_list_alpha)(normalizer,corpus_chunk) for corpus_chunk in par_corpus)\n   746                                                           \n   747                                                           # TODO: check if the reduce method is not better\n   748         1          9.0      9.0      0.0                  chain_list = []\n   749         1         19.0     19.0      0.0                  alphabet = Counter()\n   750         5         30.0      6.0      0.0                  for chain_l, alpha in result:\n   751         4     101143.0  25285.8      0.3                      chain_list += chain_l\n   752         4       1192.0    298.0      0.0                      alphabet += alpha\n   753         1        272.0    272.0      0.0                  print(f\"Normalize & Alphabet computed in {datetime.now()-start}\")\n   754                                                           \n   755                                                           # TODO: Add resume feature\n   756         1          5.0      5.0      0.0                  merges = []\n   757         1          5.0      5.0      0.0                  vocabulary = alphabet\n   758                                           \n   759         1         37.0     37.0      0.0                  print(\"Computing matrix values in parallel...\")\n   760         1          8.0      8.0      0.0                  start = datetime.now()\n   761                                                           \n   762         1         40.0     40.0      0.0                  par_chain = parallel_chain(chain_list, self.cpu_count, overlap=1)\n   763                                                           \n   764                                                           \n   765         1    4429198.0 4429198.0     13.8                  result = parallel_pool(delayed(self.find_best_pair)(chain_chunk) for chain_chunk in par_chain)\n   766                                                                               \n   767         1      27192.0  27192.0      0.1                  pairs = reduce(operator.add, result)\n   768         1       1638.0   1638.0      0.0                  pairs = pairs.most_common()\n   769         1        449.0    449.0      0.0                  print(f\"Matrix values computed in {datetime.now()-start}\")\n   770                                           \n   771         1     515938.0 515938.0      1.6                  cl_chain = \"[SEP] \"+\" \".join(chain_list)+\" [SEP]\"\n   772         1        307.0    307.0      0.0                  encode = {k:i for i,(k,v) in enumerate(alphabet.most_common())}\n   773         1         89.0     89.0      0.0                  decode = {i:k for k,i in encode.items()}\n   774                                                           \n   775         1          6.0      6.0      0.0                  voc_len = len(encode)\n   776         1          6.0      6.0      0.0                  special_tokens_len = 0 if special_tokens == None else len(special_tokens)\n   777                                                           \n   778         1        135.0    135.0      0.0                  print(f\"Alphabet Size: {voc_len}\")\n   779                                                           \n   780         1         15.0     15.0      0.0                  if vocab_size<0:\n   781         1          7.0      7.0      0.0                      voc_final_length = voc_len + abs(vocab_size)\n   782                                                           else:\n   783                                                               voc_final_length = vocab_size - special_tokens_len\n   784                                                               \n   785                                                           # TODO: this can yield an error if vocab_size is smaller than alphabet. Fix to make vocab_size = max(vocab_size, len(alphabet)), and don't launch matrix construction in that case.\n   786         1         11.0     11.0      0.0                  if sparse:\n   787         1       6464.0   6464.0      0.0                      data, rows, columns = extract_drc(pairs,encode)\n   788         1       3022.0   3022.0      0.0                      voc_matrix = coo_matrix((np.array(data), (np.array(rows),np.array(columns))), shape=(voc_final_length, voc_final_length), dtype=int)\n   789                                           \n   790                                                           else:\n   791                                                               voc_matrix = np.zeros((voc_final_length, voc_final_length), dtype=int)\n   792                                                               for (row,column),value in pairs:\n   793                                                                   voc_matrix[encode[row], encode[column]] = value\n   794                                                           \n   795         1         10.0     10.0      0.0                  delta_voc = voc_final_length - voc_len\n   796         1          8.0      8.0      0.0                  best_pair = \"init\"\n   797         1         14.0     14.0      0.0                  pair_count = \"---\"\n   798         1          9.0      9.0      0.0                  new_i = voc_len\n   799                                           \n   800                                           \n   801                                           \n   802                                                           # TODO: remove SparseEfficiencyWarning\n   803                                           \n   804         1      47891.0  47891.0      0.1                  t = trange(delta_voc, disable = not progress_bar)\n   805                                           \n   806                                                           \n   807         5      12973.0   2594.6      0.0                  for i in t:\n   808         4       3398.0    849.5      0.0                      t.set_description(f\"Pair: {best_pair}, {pair_count}\")\n   809         4        709.0    177.2      0.0                      t.refresh()\n   810                                           \n   811         4         28.0      7.0      0.0                      if sparse:\n   812         4        108.0     27.0      0.0                          max_i = voc_matrix.data.argmax()\n   813         4         33.0      8.2      0.0                          pair_row = voc_matrix.row[max_i]\n   814         4         24.0      6.0      0.0                          pair_col = voc_matrix.col[max_i]\n   815         4         24.0      6.0      0.0                          pair_count = voc_matrix.data[max_i]\n   816                                                               else:\n   817                                                                   pair_row,pair_col = np.unravel_index(np.argmax(voc_matrix, axis=None), voc_matrix.shape)\n   818                                                                   pair_count = voc_matrix[pair_row,pair_col]\n   819                                                               \n   820         4         30.0      7.5      0.0                      if pair_count == 0:\n   821                                                                   break\n   822                                           \n   823         4        115.0     28.8      0.0                      best_pair = (decode[pair_row], decode[pair_col])\n   824         4         27.0      6.8      0.0                      best_pair_string = \" \".join(best_pair)\n   825         4         27.0      6.8      0.0                      merges.append(best_pair_string)\n   826         4         25.0      6.2      0.0                      best_pair_string_voc = \"\".join(best_pair)\n   827         4       1044.0    261.0      0.0                      re_voc_l = \"(\"+\"|\".join([\" \"+k+\" \" for k in encode.keys()]+[\"\\[SEP\\] \",\"\\[SEP_i\\] \"])+\")\"\n   828         4        731.0    182.8      0.0                      re_voc_r = \"(\"+\"|\".join([\" \"+k+\" \" for k in encode.keys()]+[\" \\[SEP\\]\",\" \\[SEP_i\\]\"])+\")\"\n   829                                           \n   830                                           \n   831         8    2074652.0 259331.5      6.5                      chain_chunks = list(\n   832         4    1342415.0 335603.8      4.2                          separate_chain(cl_chain.split(), self.cpu_count, list(best_pair)))\n   833         4   14423860.0 3605965.0     45.0                      result = parallel_pool(delayed(self.findall_contexts)(chain_chunk,best_pair_string,re_voc_l,re_voc_r) for chain_chunk in chain_chunks)\n   834         4      66528.0  16632.0      0.2                      merge_context = reduce(operator.add, result)\n   835                                                                   \n   836                                                               # TODO: Maybe this can be done with a direct Counter\n   837                                                               \n   838                                                               ## Test to do implement direct counter\n   839                                                               # merge_context_count_l = Counter()\n   840                                                               # merge_context_count_r = Counter()\n   841                                                               # for l,r in merge_context:\n   842                                                               #     if \"[SEP]\" not in l:\n   843                                                               #         merge_context_count_l[encode[l.strip()]] += 1\n   844                                                               #     if \"[SEP]\" not in r:\n   845                                                               #         merge_context_count_r[encode[r.strip()]] += 1\n   846         4     201949.0  50487.2      0.6                      l_context = [l for l,r in merge_context]\n   847         4     107966.0  26991.5      0.3                      merge_context_count_l = Counter(l_context)\n   848         4      11035.0   2758.8      0.0                      merge_context_count_l = {encode[k.strip()]:v for k,v in merge_context_count_l.most_common() if \"[SEP]\" not in k}\n   849                                                               \n   850         4     184373.0  46093.2      0.6                      r_context = [r for l,r in merge_context]\n   851         4     105632.0  26408.0      0.3                      merge_context_count_r = Counter(r_context)\n   852         4        375.0     93.8      0.0                      merge_context_count_r = {encode[k.strip()]:v for k,v in merge_context_count_r.most_common() if \"[SEP]\" not in k}\n   853                                                               \n   854                                                               \n   855         4         21.0      5.2      0.0                      if sparse:\n   856                                                                   # Convert matrix to CSR or LIL, for item attribution and arithmetic \n   857         4         20.0      5.0      0.0                          if sparse_mode == \"csr\":\n   858         4       1846.0    461.5      0.0                              voc_matrix = voc_matrix.tocsr()\n   859                                                                   else:\n   860                                                                       voc_matrix = voc_matrix.tolil()\n   861                                                               \n   862       373       2132.0      5.7      0.0                      for row,key in merge_context_count_l.items():\n   863       369     166661.0    451.7      0.5                          voc_matrix[row,new_i] = key\n   864                                                                   \n   865       375       2076.0      5.5      0.0                      for column,key in merge_context_count_r.items():\n   866       371     162437.0    437.8      0.5                          voc_matrix[new_i,column] = key\n   867                                           \n   868                                                               # Correct previous counts\n   869                                                               \n   870                                                               # compute #(l,r)-(l,r)\n   871         4     136675.0  34168.8      0.4                      pair_pair_count = len(re.findall(\" \"+best_pair_string+\" \"+best_pair_string+\" \", cl_chain, overlapped=False))\n   872                                                               # remove #(l,r)-(l,r) from (l,r)-l\n   873         4       1065.0    266.2      0.0                      voc_matrix[new_i,pair_row] -= pair_pair_count\n   874                                                               # remove #(l,r)-(l,r) from r-(l,r)\n   875         4        588.0    147.0      0.0                      voc_matrix[pair_col,new_i] -= pair_pair_count\n   876                                                               # remove #(l,r)-(l,r) from r-l\n   877         4        520.0    130.0      0.0                      voc_matrix[pair_col,pair_row] -= pair_pair_count\n   878                                                               # substract (l,r)- from r-\n   879         4       5365.0   1341.2      0.0                      voc_matrix[pair_col,:new_i] -= voc_matrix[new_i,:new_i]\n   880                                                               # substract -(l,r)- from -l\n   881         4       4689.0   1172.2      0.0                      voc_matrix[:new_i,pair_row] -= voc_matrix[:new_i,new_i]\n   882                                                               \n   883                                                               # set l-r to 0\n   884         4        457.0    114.2      0.0                      voc_matrix[pair_row,pair_col] = 0\n   885                                                               # register #(l,r)-(l,r)\n   886         4       1903.0    475.8      0.0                      voc_matrix[new_i,new_i] = pair_pair_count\n   887                                                               \n   888         4         21.0      5.2      0.0                      if sparse:\n   889                                                                   # Convert matrix back to COO, to restart the loop\n   890         4        605.0    151.2      0.0                          voc_matrix = voc_matrix.tocoo()\n   891                                                               \n   892         4         24.0      6.0      0.0                      best_pair_string_voc = \"\".join(best_pair)\n   893         4         21.0      5.2      0.0                      encode[best_pair_string_voc] = new_i\n   894         4         18.0      4.5      0.0                      decode[new_i] = best_pair_string_voc\n   895         4         19.0      4.8      0.0                      new_i += 1\n   896         4    1297557.0 324389.2      4.0                      cl_chain = agglutinate_chain(best_pair_string.split(),cl_chain)\n   897                                           \n   898         4         32.0      8.0      0.0                      if saveQ == True:\n   899                                                                   voc_partial_len = voc_len + special_tokens_len + i + 1\n   900                                                                   if voc_partial_len % save_step == 0:\n   901                                                                       \n   902                                                                       if sparse:\n   903                                                                           freq_values = voc_matrix.sum(axis=1).T.tolist()[0]\n   904                                                                       else:\n   905                                                                           freq_values = voc_matrix.sum(axis=1).T.tolist()\n   906                                                                       vocabulary = {decode[i]:v for i,v in enumerate(freq_values) if v>0} # Make sure dimension of matrix and size of voc coincide\n   907                                                                       vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)\n   908                                                                       \n   909                                                                       if special_tokens != None:\n   910                                                                           vocabulary = vocabulary + [(token,0) for token in special_tokens]\n   911                                           \n   912                                                                       self.merges = merges\n   913                                                                       self.encode = {k:i for i,(k,v) in enumerate(vocabulary)}\n   914                                                                       self.freq = dict(vocabulary)\n   915                                                                       self.alpha = dict(alphabet)\n   916                                                                       step_path = self.path / str(voc_partial_len)\n   917                                                                       self.save(step_path)\n   918                                                                       print(f\"Intermediate vocabulary saved to {step_path}\")\n   919                                                                       \n   920                                                                       \n   921                                                   else:\n   922                                                       chain_list, alphabet = self.chain_list_alpha(normalizer, self.corpus.train[:corpus_length], progress_bar=True)\n   923                                                   \n   924                                                       merges = []\n   925                                                       vocabulary = alphabet\n   926                                           \n   927                                                       pairs = self.find_best_pair(chain_list, progress_bar = True)\n   928                                                       pairs = pairs.most_common()\n   929                                           \n   930                                           \n   931                                                       cl_chain = \"[SEP] \"+\" \".join(chain_list)+\" [SEP]\"\n   932                                                       encode = {k:i for i,(k,v) in enumerate(alphabet.most_common())}\n   933                                                       decode = {i:k for k,i in encode.items()}\n   934                                                       \n   935                                                       voc_len = len(encode)\n   936                                                       special_tokens_len = 0 if special_tokens == None else len(special_tokens)\n   937                                                       \n   938                                                       print(f\"Alphabet Size: {voc_len}\")\n   939                                                       \n   940                                                       if vocab_size<0:\n   941                                                           voc_final_length = voc_len + abs(vocab_size)\n   942                                                       else:\n   943                                                           voc_final_length = vocab_size - special_tokens_len\n   944                                                   \n   945                                                       # TODO: this can yield an error if vocab_size is smaller than alphabet. Fix to make vocab_size = max(vocab_size, len(alphabet)), and don't launch matrix construction in that case.\n   946                                                       if sparse:\n   947                                                           data, rows, columns = extract_drc(pairs,encode)\n   948                                                           voc_matrix = coo_matrix((np.array(data), (np.array(rows),np.array(columns))), shape=(voc_final_length, voc_final_length), dtype=int)\n   949                                           \n   950                                                       else:\n   951                                                           voc_matrix = np.zeros((voc_final_length, voc_final_length), dtype=int)\n   952                                                           for (row,column),value in pairs:\n   953                                                               voc_matrix[encode[row], encode[column]] = value\n   954                                                       \n   955                                                       delta_voc = voc_final_length - voc_len\n   956                                                       best_pair = \"init\"\n   957                                                       pair_count = \"---\"\n   958                                                       new_i = voc_len\n   959                                           \n   960                                                       # TODO: remove SparseEfficiencyWarning\n   961                                           \n   962                                                       t = trange(delta_voc, disable = not progress_bar)\n   963                                           \n   964                                           \n   965                                                       profile = Profiler()\n   966                                                       profile.start()\n   967                                                       \n   968                                                       for i in t:\n   969                                                           t.set_description(f\"Pair: {best_pair}, {pair_count}\")\n   970                                                           t.refresh()\n   971                                           \n   972                                                           if sparse:\n   973                                                               max_i = voc_matrix.data.argmax()\n   974                                                               pair_row = voc_matrix.row[max_i]\n   975                                                               pair_col = voc_matrix.col[max_i]\n   976                                                               pair_count = voc_matrix.data[max_i]\n   977                                                           else:\n   978                                                               pair_row,pair_col = np.unravel_index(np.argmax(voc_matrix, axis=None), voc_matrix.shape)\n   979                                                               pair_count = voc_matrix[pair_row,pair_col]\n   980                                                           \n   981                                                           if pair_count == 0:\n   982                                                               break\n   983                                           \n   984                                                           best_pair = (decode[pair_row], decode[pair_col])\n   985                                                           best_pair_string = \" \".join(best_pair)\n   986                                                           merges.append(best_pair_string)\n   987                                                           best_pair_string_voc = \"\".join(best_pair)\n   988                                                           re_voc_l = \"(\"+\"|\".join([\" \"+k+\" \" for k in encode.keys()]+[\"\\[SEP\\] \",\"\\[SEP_i\\] \"])+\")\"\n   989                                                           re_voc_r = \"(\"+\"|\".join([\" \"+k+\" \" for k in encode.keys()]+[\" \\[SEP\\]\",\" \\[SEP_i\\]\"])+\")\"\n   990                                                           if parallel:\n   991                                                               chain_chunks = list(\n   992                                                                   separate_chain(cl_chain.split(), self.cpu_count, list(best_pair)))\n   993                                                               result = util.multiprocessing(\n   994                                                                   partial(self.findall_contexts,best_pair_string=best_pair_string,re_voc_l=re_voc_l,re_voc_r=re_voc_r),chain_chunks,\n   995                                                                   cores = self.cpu_count\n   996                                                                   )\n   997                                                               \n   998                                                               merge_context = reduce(operator.add, result)\n   999                                                           else:\n  1000                                                               merge_context = re.findall(re_voc_l+best_pair_string+re_voc_r, cl_chain, overlapped=True)\n  1001                                                           merge_context_count_l = Counter()\n  1002                                                           merge_context_count_r = Counter()\n  1003                                                           for l,r in merge_context:\n  1004                                                               if \"[SEP]\" not in l:\n  1005                                                                   merge_context_count_l[encode[l.strip()]] += 1\n  1006                                                               if \"[SEP]\" not in r:\n  1007                                                                   merge_context_count_r[encode[r.strip()]] += 1\n  1008                                                           \n  1009                                                           if sparse:\n  1010                                                               # Convert matrix to CSR or LIL, for item attribution and arithmetic \n  1011                                                               if sparse_mode == \"csr\":\n  1012                                                                   voc_matrix = voc_matrix.tocsr()\n  1013                                                               else:\n  1014                                                                   voc_matrix = voc_matrix.tolil()\n  1015                                                           \n  1016                                                           for row,key in merge_context_count_l.items():\n  1017                                                               voc_matrix[row,new_i] = key\n  1018                                                               \n  1019                                                           for column,key in merge_context_count_r.items():\n  1020                                                               voc_matrix[new_i,column] = key\n  1021                                           \n  1022                                                           # Correct previous counts\n  1023                                                           \n  1024                                                           # compute #(l,r)-(l,r)\n  1025                                                           pair_pair_count = len(re.findall(\" \"+best_pair_string+\" \"+best_pair_string+\" \", cl_chain, overlapped=False))\n  1026                                                           # remove #(l,r)-(l,r) from (l,r)-l\n  1027                                                           voc_matrix[new_i,pair_row] -= pair_pair_count\n  1028                                                           # remove #(l,r)-(l,r) from r-(l,r)\n  1029                                                           voc_matrix[pair_col,new_i] -= pair_pair_count\n  1030                                                           # remove #(l,r)-(l,r) from r-l\n  1031                                                           voc_matrix[pair_col,pair_row] -= pair_pair_count\n  1032                                                           # substract (l,r)- from r-\n  1033                                                           voc_matrix[pair_col,:new_i] -= voc_matrix[new_i,:new_i]\n  1034                                                           # substract -(l,r)- from -l\n  1035                                                           voc_matrix[:new_i,pair_row] -= voc_matrix[:new_i,new_i]\n  1036                                                           \n  1037                                                           # set l-r to 0\n  1038                                                           voc_matrix[pair_row,pair_col] = 0\n  1039                                                           # register #(l,r)-(l,r)\n  1040                                                           voc_matrix[new_i,new_i] = pair_pair_count\n  1041                                                           \n  1042                                                           if sparse:\n  1043                                                               # Convert matrix back to COO, to restart the loop\n  1044                                                               voc_matrix = voc_matrix.tocoo()\n  1045                                                           \n  1046                                                           best_pair_string_voc = \"\".join(best_pair)\n  1047                                                           encode[best_pair_string_voc] = new_i\n  1048                                                           decode[new_i] = best_pair_string_voc\n  1049                                                           new_i += 1\n  1050                                                           cl_chain = agglutinate_chain(best_pair_string.split(),cl_chain)\n  1051                                           \n  1052                                                           if saveQ == True:\n  1053                                                               voc_partial_len = voc_len + special_tokens_len + i + 1\n  1054                                                               if voc_partial_len % save_step == 0:\n  1055                                                                   \n  1056                                                                   if sparse:\n  1057                                                                       freq_values = voc_matrix.sum(axis=1).T.tolist()[0]\n  1058                                                                   else:\n  1059                                                                       freq_values = voc_matrix.sum(axis=1).T.tolist()\n  1060                                                                   vocabulary = {decode[i]:v for i,v in enumerate(freq_values) if v>0} # Make sure dimension of matrix and size of voc coincide\n  1061                                                                   vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)\n  1062                                                                   \n  1063                                                                   if special_tokens != None:\n  1064                                                                       vocabulary = vocabulary + [(token,0) for token in special_tokens]\n  1065                                           \n  1066                                                                   self.merges = merges\n  1067                                                                   self.encode = {k:i for i,(k,v) in enumerate(vocabulary)}\n  1068                                                                   self.freq = dict(vocabulary)\n  1069                                                                   self.alpha = dict(alphabet)\n  1070                                                                   step_path = self.path / str(voc_partial_len)\n  1071                                                                   self.save(step_path)\n  1072                                                                   print(f\"Intermediate vocabulary saved to {step_path}\")\n  1073                                           \n  1074         1          7.0      7.0      0.0          if sparse:\n  1075         1        346.0    346.0      0.0              freq_values = voc_matrix.sum(axis=1).T.tolist()[0]\n  1076                                                   else:\n  1077                                                       freq_values = voc_matrix.sum(axis=1).T.tolist()\n  1078         1        206.0    206.0      0.0          vocabulary = {decode[i]:v for i,v in enumerate(freq_values) if v>0} # Make sure dimension of matrix and size of voc coincide\n  1079         1      16524.0  16524.0      0.1          vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)\n  1080                                                   \n  1081         1          8.0      8.0      0.0          if special_tokens != None:\n  1082         1         14.0     14.0      0.0              vocabulary = vocabulary + [(token,0) for token in special_tokens]\n  1083                                                   \n  1084         1          8.0      8.0      0.0          self.merges = merges\n  1085         1        144.0    144.0      0.0          self.encode = {k:i for i,(k,v) in enumerate(vocabulary)}\n  1086         1         38.0     38.0      0.0          self.freq = dict(vocabulary)\n  1087         1        115.0    115.0      0.0          self.alpha = dict(alphabet.most_common())\n  1088                                           \n  1089         1         93.0     93.0      0.0          self.decode = {i:k for k,i in self.encode.items()}\n  1090                                                   \n  1091         1          7.0      7.0      0.0          self.len = len(vocabulary)     \n  1092         1         25.0     25.0      0.0          self.freq_mass = sum(self.freq.values())\n  1093         1        145.0    145.0      0.0          self.prob = {k:v/self.freq_mass for k,v in self.freq.items()}\n  1094                                           \n  1095         1        207.0    207.0      0.0          print(\"Vocabulary built\")\n  1096                                                   \n  1097         1         10.0     10.0      0.0          if save == True:\n  1098                                                       self.save()\n  1099                                                       print(f\"Vocabulary saved to {self.path}\")"

In [40]:
n = 17500
print(profiler_result[n:n+2000])

423860.0 3605965.0     45.0                      result = parallel_pool(delayed(self.findall_contexts)(chain_chunk,best_pair_string,re_voc_l,re_voc_r) for chain_chunk in chain_chunks)
   834         4      66528.0  16632.0      0.2                      merge_context = reduce(operator.add, result)
   835                                                                   
   836                                                               # TODO: Maybe this can be done with a direct Counter
   837                                                               
   838                                                               ## Test to do implement direct counter
   839                                                               # merge_context_count_l = Counter()
   840                                                               # merge_context_count_r = Counter()
   841                                                               # for l,r in merge_context:
   842               

In [9]:
semiotic.vocab.build_par(vocab_size = -10, parallel=True)

Normalize & Alphabet in parallel...
Normalize & Alphabet computed in 0:00:06.025894
Computing matrix values in parallel...
Matrix values computed in 0:00:04.092233
Alphabet Size: 493


/usr/local/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])



Vocabulary built
